In [1]:
import os
import sys
import time

import numpy as np
import torch
import pandas as pd

import matplotlib.pyplot as plt

import scipy.io.wavfile as wav

import importlib
import librosa
import soundfile as sf

import torchnet.meter as tnt
from collections import OrderedDict

from IPython.display import clear_output
from scipy.io.wavfile import write
import IPython

sys.path.append(os.path.join(os.getcwd(), '..'))
sys.path.append(os.path.join(os.getcwd(), '../models/'))
sys.path.append(os.path.join(os.getcwd(), '../datasets/'))

import ai8x

from types import SimpleNamespace
from msnoise import MSnoise
import msnoise

from mixedkws import MixedKWS
import mixedkws

In [2]:
trained_checkpoint_path = os.path.join("/home/merveeyuboglu/Github/ai8x-synthesis/trained/","ai85-kws20_v3-qat8.pth.tar")
mod = importlib.import_module("ai85net-kws20-v3")
dataset = importlib.import_module("kws20")

In [3]:
classes = ['up', 'down', 'left', 'right', 'stop', 'go', 'yes', 'no', 'on', 'off', 'one',
           'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'zero', 'unknown']

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Working with device:", device)

ai8x.set_device(device=85, simulate=False, round_avg=False)
# qat_policy = {'start_epoch': 10, 'weight_bits': 8, 'bias_bits': 8}

Working with device: cuda
Configuring device: MAX78000, simulate=False.


In [5]:
model = mod.AI85KWS20Netv3(num_classes=len(classes), num_channels=128, dimensions=(128, 1), bias=False, 
                           quantize_activation=False)

checkpoint = torch.load(trained_checkpoint_path)

state_dict = checkpoint['state_dict']
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    if k.startswith('module.'):
        k = k[7:]
    new_state_dict[k] = v
checkpoint['state_dict'] = new_state_dict

model.load_state_dict(checkpoint['state_dict'], strict=False)

ai8x.update_model(model)

In [7]:
sn = SimpleNamespace()
sn.truncate_testset = False
sn.act_mode_8bit = False

# args = ('/data_ssd', Namespace(act_mode_8bit=False, activation_stats=[], avg_pool_rounding=False, batch_size=256, cnn='ai85kws20netv3', collate_fn=None, compress='policies/schedule_kws20.yaml', cpu=False, csv_prefix=None, data='/data_ssd', dataset='KWS_20', datasets_fn=<function KWS_20_get_datasets at 0x7f917f0f2040>, deterministic=True, device='cuda', dimensions=(128, 128, 1), display_confusion=True, display_embedding=False, display_prcurves=False, earlyexit_lossweights=None, earlyexit_thresholds=None, effective_test_size=1.0, effective_train_size=1.0, effective_valid_size=1.0, epochs=200, evaluate=False, extras=None, generate_sample=None, gpus=None, kd_distill_wt=0.5, kd_pretrained=False, kd_resume='', kd_start_epoch=0, kd_student_wt=0.5, kd_teacher=None, kd_teacher_wt=0.0, kd_temp=1.0, kernel_stats=False, labels=('up', 'down', 'left', 'right', 'stop', 'go', 'yes', 'no', 'on', 'off', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'zero', 'UNKNOWN'), load_model_path='', load_serialized=False, log_params_histograms=False, lr=0.001, masks_sparsity=False, momentum=0.9, name=None, nas=False, nas_kd_resume_from='', nas_policy='', nas_stage_transition_list=None, no_tensorboard=True, num_best_scores=1, num_classes=21, obj_detection=False, obj_detection_params=None, optimizer='Adam', out_fold_ratio=1, output_dir='logs', pretrained=False, print_freq=10, qat_policy='policies/qat_policy.yaml', qe_bits_accum=32, qe_bits_acts=8, qe_bits_wts=8, qe_calibration=None, qe_clip_acts=<ClipMode.NONE: 0>, qe_clip_n_stds=None, qe_config_file=None, qe_convert_pytorch=False, qe_dynamic=False, qe_mode=<LinearQuantMode.SYMMETRIC: 1>, qe_mode_acts=None, qe_mode_wts=None, qe_no_clip_layers=[], qe_no_quant_layers=[], qe_per_channel=False, qe_pytorch_backend='fbgemm', qe_save_fp_weights=False, qe_scale_approx_bits=None, qe_stats_file=None, quantize_eval=False, regression=False, reset_optimizer=False, resumed_checkpoint_path='', seed=None, sensitivity=None, sensitivity_range=[0.0, 0.95, 0.05], shap=0, show_train_accuracy='last_batch', sparsity_perf=False, summary=None, summary_filename='model', tblog=False, thinnify=False, truncate_testset=False, use_bias=False, validation_split=0.1, visualize_fn=<function visualize_data at 0x7f8fface3790>, weight_decay=0.0, workers=4))

_, test_dataset = dataset.KWS_35_get_unquantized_datasets( ('/data_ssd', sn), load_train=False, load_test=True)


Processing test...
test set: 12616 elements
Class backward (# 0): 164 elements
Class bed (# 1): 194 elements
Class bird (# 2): 208 elements
Class cat (# 3): 205 elements
Class dog (# 4): 227 elements
Class down (# 5): 385 elements
Class eight (# 6): 372 elements
Class five (# 7): 407 elements
Class follow (# 8): 161 elements
Class forward (# 9): 166 elements
Class four (# 10): 374 elements
Class go (# 11): 365 elements
Class happy (# 12): 231 elements
Class house (# 13): 212 elements
Class learn (# 14): 155 elements
Class left (# 15): 377 elements
Class marvin (# 17): 211 elements
Class nine (# 18): 385 elements
Class no (# 19): 365 elements
Class off (# 20): 359 elements
Class on (# 21): 388 elements
Class one (# 22): 385 elements
Class right (# 23): 366 elements
Class seven (# 24): 395 elements
Class sheila (# 25): 224 elements
Class six (# 26): 380 elements
Class stop (# 27): 369 elements
Class three (# 28): 354 elements
Class tree (# 29): 180 elements
Class two (# 30): 380 element

In [6]:
sn = SimpleNamespace()
sn.truncate_testset = False
sn.act_mode_8bit = False

# args = ('/data_ssd', Namespace(act_mode_8bit=False, activation_stats=[], avg_pool_rounding=False, batch_size=256, cnn='ai85kws20netv3', collate_fn=None, compress='policies/schedule_kws20.yaml', cpu=False, csv_prefix=None, data='/data_ssd', dataset='KWS_20', datasets_fn=<function KWS_20_get_datasets at 0x7f917f0f2040>, deterministic=True, device='cuda', dimensions=(128, 128, 1), display_confusion=True, display_embedding=False, display_prcurves=False, earlyexit_lossweights=None, earlyexit_thresholds=None, effective_test_size=1.0, effective_train_size=1.0, effective_valid_size=1.0, epochs=200, evaluate=False, extras=None, generate_sample=None, gpus=None, kd_distill_wt=0.5, kd_pretrained=False, kd_resume='', kd_start_epoch=0, kd_student_wt=0.5, kd_teacher=None, kd_teacher_wt=0.0, kd_temp=1.0, kernel_stats=False, labels=('up', 'down', 'left', 'right', 'stop', 'go', 'yes', 'no', 'on', 'off', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'zero', 'UNKNOWN'), load_model_path='', load_serialized=False, log_params_histograms=False, lr=0.001, masks_sparsity=False, momentum=0.9, name=None, nas=False, nas_kd_resume_from='', nas_policy='', nas_stage_transition_list=None, no_tensorboard=True, num_best_scores=1, num_classes=21, obj_detection=False, obj_detection_params=None, optimizer='Adam', out_fold_ratio=1, output_dir='logs', pretrained=False, print_freq=10, qat_policy='policies/qat_policy.yaml', qe_bits_accum=32, qe_bits_acts=8, qe_bits_wts=8, qe_calibration=None, qe_clip_acts=<ClipMode.NONE: 0>, qe_clip_n_stds=None, qe_config_file=None, qe_convert_pytorch=False, qe_dynamic=False, qe_mode=<LinearQuantMode.SYMMETRIC: 1>, qe_mode_acts=None, qe_mode_wts=None, qe_no_clip_layers=[], qe_no_quant_layers=[], qe_per_channel=False, qe_pytorch_backend='fbgemm', qe_save_fp_weights=False, qe_scale_approx_bits=None, qe_stats_file=None, quantize_eval=False, regression=False, reset_optimizer=False, resumed_checkpoint_path='', seed=None, sensitivity=None, sensitivity_range=[0.0, 0.95, 0.05], shap=0, show_train_accuracy='last_batch', sparsity_perf=False, summary=None, summary_filename='model', tblog=False, thinnify=False, truncate_testset=False, use_bias=False, validation_split=0.1, visualize_fn=<function visualize_data at 0x7f8fface3790>, weight_decay=0.0, workers=4))

_, test_dataset = dataset.KWS_20_get_datasets( ('/data_ssd', sn), load_train=False, load_test=True)
_, test_noise = msnoise.MSnoise_get_datasets(('/data_ssd', sn), load_train=False, load_test=True)

No key `stretch` in input augmentation dictionary! Using defaults: [Min: 0.8, Max: 1.3]

Processing test...
test set: 33015 elements
Class up (# 31): 1275 elements
Class down (# 5): 1218 elements
Class left (# 15): 1236 elements
Class right (# 23): 1188 elements
Class stop (# 27): 1233 elements
Class go (# 11): 1206 elements
Class yes (# 34): 1257 elements
Class no (# 19): 1215 elements
Class on (# 21): 1188 elements
Class off (# 20): 1206 elements
Class one (# 22): 1197 elements
Class two (# 30): 1272 elements
Class three (# 28): 1215 elements
Class four (# 10): 1200 elements
Class five (# 7): 1335 elements
Class six (# 26): 1182 elements
Class seven (# 24): 1218 elements
Class eight (# 6): 1224 elements
Class nine (# 18): 1224 elements
Class zero (# 35): 1254 elements
Class UNKNOWN: 8472 elements
torch.Size([56253, 128, 128])


Class AirConditioner, 0
Number of elements in class AirConditioner: 2513
Class AirportAnnouncements, 1
Number of elements in class AirportAnnouncements: 2514


In [7]:
originals = list(range(0, len(test_dataset), 3))
original_testdataset  = torch.utils.data.Subset(test_dataset, originals)
original_test_loader = torch.utils.data.DataLoader(original_testdataset, batch_size=256)
test_loader = original_test_loader

In [5]:
#test_dataset.data = test_dataset.data[:1000]
#test_dataset.targets = test_dataset.targets[:1000]

In [6]:
#test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=256)

In [8]:
class_dict = {}
class_nums = test_noise.targets.unique()

for idx, i in enumerate(test_noise.classes):
    class_dict[i] = idx

noise_classes = {}
start = 0
end = 0

for i in test_noise.classes:
    
    val = class_dict[i]
    val = list(test_noise.targets).count(val)
    
    if val == 0:
        continue

    end += val
    noise_classes[i] = (start, end)

    start = end

In [9]:
noise_classes.keys()

dict_keys(['AirConditioner', 'AirportAnnouncements', 'Babble', 'Bus', 'Car', 'CopyMachine', 'ShuttingDoor', 'Typing', 'VacuumCleaner'])

In [10]:
def evaluate(db, noise, noise_kind):

    if (noise == True) and (noise_kind == None):
        print('Noise kind is not specified. Noise will not be applied.')
        noise = False

    model.eval()
    model.to(device)
    classerr = tnt.ClassErrorMeter(accuracy=True, topk=(1, min(len(classes), 5)))
    outputs_all = np.zeros((len(test_dataset), 21))
    targets_all = np.zeros((len(test_dataset)))

    if (noise == True) and (noise_kind != 'WhiteNoise'):
        low, high = noise_classes[noise_kind]

    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):

            if noise == True:
                if noise_kind == 'WhiteNoise':

                    mean = 0
                    std = 1 
                    random_noise = np.random.normal(mean, std, inputs.shape)
                    random_noise = torch.tensor(random_noise, dtype = torch.float32)
                else:
                    n_noise = np.random.randint(low, high, inputs.shape[0])
                    random_noise = torch.tensor(test_noise.data[n_noise], dtype = torch.float32)

                inputs = MixedKWS.snr_mixer(inputs, random_noise, db)

            inputs = inputs.to(device)
            targets = targets.to(device)
            outputs = model(inputs)
            classerr.add(outputs, targets)

            print("Batch: [",batch_idx*256 ,"/", len(test_dataset),"]")
            acc = classerr.value()[0]
            print("Accuracy: ", acc)
            outputs_all[batch_idx*256:batch_idx*256+len(inputs)] = outputs.cpu().numpy()
            targets_all[batch_idx*256:batch_idx*256+len(inputs)] = targets.cpu().numpy()
            
    print("Total Accuracy: ", acc)
    return acc

In [11]:
def snr_testing(noise = False, noise_kind = None):

    if noise == False:
        db = None

    time_s = time.time()
    print("Started evaluating :", trained_checkpoint_path)

    if noise == False:
        accuracies = np.zeros(1)
        snr = np.zeros(1)

        accuracies[0] = evaluate(db, noise, noise_kind)
        snr[0] = None
        
        dur = time.time() - time_s
        print(f'Finished in {dur:.3f} seconds.')
        return accuracies, snr
    
    accuracies = np.zeros(4)
    snr = np.zeros(4)

    for idx, db in enumerate([-5, 0, 10, 20]):
        print("Evaluating SNR levels of", db)
        accuracies[idx] = evaluate(db, noise, noise_kind)
        snr[idx] = db
    
    dur = time.time() - time_s
    print(f'Finished in {dur:.3f} seconds.')
    return accuracies, snr

In [12]:
def benchmark(noise_kinds):

    num = len(noise_kinds) + 1

    snr = np.zeros((num, 4))
    accuracies = np.zeros((num, 4))

    for idx, n in enumerate(noise_kinds):

        print(f'{n} Noise Evaluation')

        accuracies[idx], snr[idx] = snr_testing(noise = True, noise_kind = n)


    accuracies[-1], snr[-1] = snr_testing(noise= False)


    return accuracies, snr

In [13]:
noise_list = noise_classes.keys()

In [14]:
accuracies_w, snr_w = benchmark(noise_kinds = noise_list)

AirConditioner Noise Evaluation
Started evaluating : /home/merveeyuboglu/Github/ai8x-synthesis/trained/ai85-kws20_v3-qat8.pth.tar
Evaluating SNR levels of -5


/tmp/ipykernel_3441499/4205357698.py:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  random_noise = torch.tensor(test_noise.data[n_noise], dtype = torch.float32)


Batch: [ 0 / 12616 ]
Accuracy:  0.0
Batch: [ 256 / 12616 ]
Accuracy:  0.78125
Batch: [ 512 / 12616 ]
Accuracy:  0.520833333333337
Batch: [ 768 / 12616 ]
Accuracy:  0.78125
Batch: [ 1024 / 12616 ]
Accuracy:  0.7031250000000044
Batch: [ 1280 / 12616 ]
Accuracy:  0.9765625
Batch: [ 1536 / 12616 ]
Accuracy:  0.8928571428571397
Batch: [ 1792 / 12616 ]
Accuracy:  0.830078125
Batch: [ 2048 / 12616 ]
Accuracy:  0.868055555555558
Batch: [ 2304 / 12616 ]
Accuracy:  1.1328125000000022
Batch: [ 2560 / 12616 ]
Accuracy:  1.3139204545454586
Batch: [ 2816 / 12616 ]
Accuracy:  1.46484375
Batch: [ 3072 / 12616 ]
Accuracy:  3.1850961538461564
Batch: [ 3328 / 12616 ]
Accuracy:  3.2087053571428603
Batch: [ 3584 / 12616 ]
Accuracy:  3.281250000000002
Batch: [ 3840 / 12616 ]
Accuracy:  3.1005859375
Batch: [ 4096 / 12616 ]
Accuracy:  3.0432715168806412
Total Accuracy:  3.0432715168806412
Evaluating SNR levels of 0
Batch: [ 0 / 12616 ]
Accuracy:  0.390625
Batch: [ 256 / 12616 ]
Accuracy:  1.7578125
Batch: [ 5

In [21]:
snr = [-5, 0, 10, 20]

In [22]:
csv_dict = {}

csv_dict['raw'] = accuracies_w[-1][0]

for idx, i in enumerate(noise_list):

    for idx2, j in enumerate(accuracies_w[idx]):
        name =  f'{i} {snr[idx2]}'
        csv_dict[name] = j

In [23]:
csv_dict

{'raw': 81.4,
 'AirConditioner -5': 11.899999999999999,
 'AirConditioner 0': 21.199999999999996,
 'AirConditioner 10': 48.1,
 'AirConditioner 20': 72.3,
 'AirportAnnouncements -5': 22.099999999999998,
 'AirportAnnouncements 0': 30.000000000000004,
 'AirportAnnouncements 10': 50.8,
 'AirportAnnouncements 20': 72.0,
 'Babble -5': 12.6,
 'Babble 0': 18.700000000000006,
 'Babble 10': 41.0,
 'Babble 20': 66.7,
 'Bus -5': 53.89999999999999,
 'Bus 0': 64.0,
 'Bus 10': 78.3,
 'Bus 20': 81.5,
 'Car -5': 54.400000000000006,
 'Car 0': 65.2,
 'Car 10': 78.7,
 'Car 20': 81.2,
 'CopyMachine -5': 26.8,
 'CopyMachine 0': 39.800000000000004,
 'CopyMachine 10': 59.3,
 'CopyMachine 20': 74.8,
 'ShuttingDoor -5': 45.3,
 'ShuttingDoor 0': 50.8,
 'ShuttingDoor 10': 67.5,
 'ShuttingDoor 20': 76.2,
 'Typing -5': 33.599999999999994,
 'Typing 0': 45.9,
 'Typing 10': 67.19999999999999,
 'Typing 20': 77.3,
 'VacuumCleaner -5': 2.6000000000000023,
 'VacuumCleaner 0': 7.099999999999995,
 'VacuumCleaner 10': 27.1,
 

In [24]:
current = pd.read_csv('/home/merveeyuboglu/Github/ai8x-training/codes/cr.csv', decimal=',', sep=';', index_col=0)

In [28]:
df = pd.DataFrame(csv_dict, index=["ai85net-kws20-v3"]).transpose()
new_df = pd.concat([current, df], axis =1)

In [29]:
new_df.to_csv('cr.csv', sep=';', decimal=',')

In [30]:
new_df

,ai85net-kws20-v3,ai85net-kws20-v2,ai85net-kws20-v3
raw,84.8,98.1,81.4
AirConditioner -5,15.0,55.4,11.9
AirConditioner 0,25.0,68.3,21.2
AirConditioner 10,55.9,86.1,48.1
AirConditioner 20,79.1,95.5,72.3
AirportAnnouncements -5,25.6,55.4,22.1
AirportAnnouncements 0,34.0,67.7,30.0
AirportAnnouncements 10,57.5,84.6,50.8
AirportAnnouncements 20,76.2,94.2,72.0
Babble -5,14.4,54.5,12.6
